This colab notebook creates an example campaign and populates the stats table with all the monsters available in the [5e API](https://github.com/adrpadua/5e-database) available at http://dnd5eapi.co/ . 

The basic idea of `dnd_tracker` is that you would have perhaps one notebook per adventure session, allowing you to keep a log of what happened during the campaign. 

If you've never used a notebook before, you can run each block of code sequentially by repeatedly using Shift+Enter; to run a code block and note move forward (as you would during combat to view the status without progressing), use Control+Enter.

First, we install the library and create an example campaign.

In [ ]:
!pip install git+https://github.com/klarh/dnd_tracker.git

import dnd_tracker
camp = dnd_tracker.Campaign('example')

Next, we load the data from the 5e API project.

In [ ]:
import json
import urllib

url = 'https://github.com/bagelbits/5e-database/blob/master/src/5e-SRD-Monsters.json?raw=true'
req = urllib.request.urlopen(url)
all_mobs = json.load(req)

get_mod = lambda stat: (stat - 10)//2
all_stats = ['strength', 'dexterity', 'constitution', 'intelligence', 'wisdom', 'charisma']

for mob in all_mobs:
    saves = {name[:3]: get_mod(mob[name]) for name in all_stats}

    for prof in mob['proficiencies']:
        if prof['name'].startswith('Saving Throw:'):
            short_name = prof['name'][-3:].lower()
            value = prof['value']
            saves[short_name] = value

    combat_stats = dict(
        ac=mob['armor_class'],
        dex=mob.get('dexterity', 10),
        hp=mob['hit_points'],
        saves=saves,
    )

    resistances = combat_stats['resistances'] = {}

    for type_ in mob['damage_immunities']:
        resistances[type_] = 0
    for type_ in mob['damage_resistances']:
        resistances[type_] = .5
    for type_ in mob['damage_vulnerabilities']:
        resistances[type_] = 2

    name = mob['name']
    camp.set_combat_stats(name, **combat_stats)


Now we can initiate combat and use the data that were loaded from the API!

In [ ]:
cb = camp.begin_combat('test combat')

rogue = cb.add('rogue', 23, ac=15)
cleric = cb.add('cleric', 4, ac=18)
goblin = cb.add('Goblin', 12)
ghoul = cb.add('Ghoul', 16)
ghoul2 = cb.add('Ghoul', 16)
# have the starting stats be displayed immediately
cb.show()

rogue.damage(goblin, 6)
ghoul.damage(rogue, 6)
ghoul2.damage(cleric, 4)
# goblin missed
cleric.damage(ghoul, 8, type='radiant')

rogue.damage(goblin, 3)
# ghoul missed
# ghoul2 missed
goblin.damage(rogue, 3)
cleric.damage(ghoul, 3, type='radiant')

cb.plot_damage_summary()
cb